### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('data/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('data/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
movies.describe()

,movie_id,movie,genre
count,35479,35479,35230
unique,35479,35416,2736
top,4730872,Home (2016),Drama
freq,1,4,3602


In [3]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines LumiÃ¨re (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35479 entries, 0 to 35478
Data columns (total 3 columns):
movie_id    35479 non-null object
movie       35479 non-null object
genre       35230 non-null object
dtypes: object(3)
memory usage: 831.6+ KB


In [5]:
# more work
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863866 entries, 0 to 863865
Data columns (total 4 columns):
user_id      863866 non-null object
movie_id     863866 non-null object
rating       863866 non-null int64
timestamp    863866 non-null object
dtypes: int64(1), object(3)
memory usage: 26.4+ MB


In [6]:
# and even more - open additional cells as necessary
pd.isna(reviews['rating']).sum()

0

In [7]:
#1.The number of movies in the dataset
movies.shape[0]

35479

In [8]:
#2.The number of ratings in the dataset
reviews.shape[0]

863866

In [9]:
#3. number of different genres
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

# we end up needing this later
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))
print(genres)

The number of genres is 28.
{'Adult', 'News', 'Western', 'War', 'Adventure', 'Documentary', 'Action', 'Fantasy', 'Horror', 'Romance', 'Animation', 'Biography', 'Family', 'Comedy', 'Reality-TV', 'Mystery', 'Crime', 'Film-Noir', 'Sport', 'Drama', 'Short', 'Musical', 'Thriller', 'Game-Show', 'History', 'Music', 'Sci-Fi', 'Talk-Show'}


In [10]:
#The number of unique users in the dataset
#len(np.unique(reviews.user_id))
reviews.describe(include=np.object)

,user_id,movie_id,timestamp
count,863866,863866,863866
unique,67353,35479,851107
top,23751,1454468,1484827876
freq,2875,3086,38


In [11]:
#The number missing ratings in the reviews dataset
pd.isna(reviews['rating']).sum()
#reviews.rating.isnull().sum()

0

In [12]:
#The average rating given across all ratings'
reviews.rating.mean()

7.315877693994207

In [13]:
#The minimum rating given across all ratings
reviews.rating.min()

0

In [14]:
#The maximum rating given across all ratings
reviews.rating.max()

10

In [15]:
reviews.rating.describe()

count    863866.000000
mean          7.315878
std           1.853831
min           0.000000
25%           6.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

In [16]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset':35479,
'The number of ratings in the dataset': 863866,
'The number of different genres': 28,
'The number of unique users in the dataset':67353,
'The number missing ratings in the reviews dataset':0,
'The average rating given across all ratings':7.315878,
'The minimum rating given across all ratings':0,
'The maximum rating given across all ratings':10
}



#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [17]:
#  create date function and date column
create_date = lambda x:x[-5:-1] if x[-1]==')' else np.nan
movies['date']=movies['movie'].apply(create_date)
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines LumiÃ¨re (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [18]:
# century function 
def century_dummy(val):
    if val == yr:
        return 1
    else:
        return 0
for yr in ["18","19","20"]:
    movies[str(yr)+"00's"] = movies["date"].apply(century_dummy)
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,0,0,0
1,0000010,La sortie des usines LumiÃ¨re (1895),Documentary|Short,1895,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,0,0,0


In [19]:
# dummy variables of genres
def split_genre(val):
    try:
        if val.find(g)> -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0
for g in genres:
    movies[g] = movies['genre'].apply(split_genre)

In [21]:
movies.iloc[0]

movie_id                                             0000008
movie          Edison Kinetoscopic Record of a Sneeze (1894)
genre                                      Documentary|Short
date                                                    1894
1800's                                                     0
1900's                                                     0
2000's                                                     0
Adult                                                      0
News                                                       0
Western                                                    0
War                                                        0
Adventure                                                  0
Documentary                                                1
Action                                                     0
Fantasy                                                    0
Horror                                                     0
Romance                 

In [23]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [32]:
#create date out of timestamp
import datetime
timestamp_date =lambda x:datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')

In [33]:
reviews['date_time']= reviews['timestamp'].apply(timestamp_date)
reviews.head()

,user_id,movie_id,rating,timestamp,date_time
0,1,0114508,8,1381006850,2013-10-06 02:30:50
1,2,0208092,5,1586466072,2020-04-10 02:31:12
2,2,0358273,9,1579057827,2020-01-15 08:40:27
3,2,10039344,5,1578603053,2020-01-10 02:20:53
4,2,6751668,9,1578955697,2020-01-14 04:18:17


In [ ]:
reviews.to_csv('reviews_clean',index=True)